In [1]:
import time
from neo4j import GraphDatabase

In [2]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [13]:
# Configura tu conexión a la base de datos de Neo4j Aura
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

# Diccionario de radios por categoría (en metros)
radios = {
    1: 600,
    2: 1200,
    3: 2400
}

# Diccionario para asignar el tipo de relación a cada categoría
relacion_por_categoria = {
    1: "CERCA_DE_CAT1",
    2: "CERCA_DE_CAT2",
    3: "CERCA_DE_CAT3"
}

# Inicializa el driver
driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
print("Conexión a Neo4j establecida.")

Conexión a Neo4j establecida.


In [15]:
# Iterar sobre cada categoría y crear las relaciones
with driver.session(database="neo4jads") as session:
    for cat, radio in radios.items():
        relacion = relacion_por_categoria[cat]
        print(f"\n--- Creando relaciones de tipo :{relacion} con radio de {radio}m ---")
        
        query = f"""
        CALL apoc.periodic.iterate(
            'MATCH (d:Departamento) RETURN d',
            'MATCH (p:POI)
             WHERE p.cat = $categoria AND point.distance(d.location, p.location) < $radio
             WITH d, p, point.distance(d.location, p.location) AS distancia
             CREATE (d)-[:{relacion} {{distancia_metros: distancia}}]->(p)',
            {{batchSize: 1000, parallel: false, params: {{categoria: $categoria, radio: $radio}}}}
        ) YIELD batches, total
        RETURN batches, total
        """
        
        start_time = time.time()
        result = session.run(query, categoria=cat, radio=radio).data()
        end_time = time.time()
        
        batches = result[0]['batches']
        total = result[0]['total']
        duration = round(end_time - start_time, 2)
        
        print(f"Proceso finalizado para la categoría {cat}.")
        print(f"Total de departamentos procesados: {total}.")
        print(f"Tiempo total: {duration} segundos.")

driver.close()


--- Creando relaciones de tipo :CERCA_DE_CAT1 con radio de 600m ---
Proceso finalizado para la categoría 1.
Total de departamentos procesados: 25215.
Tiempo total: 656.16 segundos.

--- Creando relaciones de tipo :CERCA_DE_CAT2 con radio de 1200m ---
Proceso finalizado para la categoría 2.
Total de departamentos procesados: 25215.
Tiempo total: 276.43 segundos.

--- Creando relaciones de tipo :CERCA_DE_CAT3 con radio de 2400m ---
Proceso finalizado para la categoría 3.
Total de departamentos procesados: 25215.
Tiempo total: 286.15 segundos.


In [18]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")
DB_NAME = "neo4jads"

# Diccionario de radios para transporte público
radios_transporte = {
    "BusStop": 400,
    "MetroStation": 800
}

# Inicializa el driver

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
print("Conexión a Neo4j local establecida.")

with driver.session(database=DB_NAME) as session:
    for tipo_transporte, radio in radios_transporte.items():
        # Define el tipo de relación de forma dinámica
        relacion = f"ACCESIBLE_POR_{tipo_transporte.upper()}"
        print(f"\n--- Creando relaciones de tipo :{relacion} con radio de {radio}m ---")
        
        # Query para conectar departamentos con el transporte público
        query = f"""
        CALL apoc.periodic.iterate(
            'MATCH (d:Departamento) RETURN d',
            'MATCH (t:{tipo_transporte})
                WHERE point.distance(d.location, t.location) < $radio
                WITH d, t, point.distance(d.location, t.location) AS distancia
                CREATE (d)-[:{relacion} {{distancia_metros: distancia}}]->(t)',
            {{batchSize: 1000, parallel: false, params: {{radio: $radio}}}}
        ) YIELD batches, total
        RETURN batches, total
        """
        
        start_time = time.time()
        result = session.run(query, radio=radio).data()
        end_time = time.time()
        
        batches = result[0]['batches']
        total = result[0]['total']
        duration = round(end_time - start_time, 2)
        
        print(f"Proceso finalizado para {tipo_transporte}.")
        print(f"Total de departamentos procesados: {total}.")
        print(f"Tiempo total: {duration} segundos.")

driver.close()

Conexión a Neo4j local establecida.

--- Creando relaciones de tipo :ACCESIBLE_POR_BUSSTOP con radio de 400m ---
Proceso finalizado para BusStop.
Total de departamentos procesados: 25215.
Tiempo total: 321.66 segundos.

--- Creando relaciones de tipo :ACCESIBLE_POR_METROSTATION con radio de 800m ---
Proceso finalizado para MetroStation.
Total de departamentos procesados: 25215.
Tiempo total: 5.43 segundos.
